### Importing Libraries

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import geopandas as gpd

### Step 1: Data Loading

In [2]:
#Calendar Table – Data Cleaning

In [3]:
# Data extraction
calendar = pd.read_csv('/kaggle/input/calender1/calendar.csv')
listings = pd.read_csv('/kaggle/input/listing1/listings.csv')
listings_extra = pd.read_csv('/kaggle/input/listing2/listings1.csv')
reviews = pd.read_csv('/kaggle/input/reviews/reviews.csv')
reviews_extra = pd.read_csv('/kaggle/input/reviews-2/reviews1.csv')
neighbourhoods = pd.read_csv('/kaggle/input/neighbourhoods/neighbourhoods.csv')
neighbourhoods_geo = gpd.read_file('/kaggle/input/neighbourhoods-geojson-1/neighbourhoods.geojson')

In [4]:
calendar.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,5456,2025-09-17,f,NaN,NaN,4,90
1,5456,2025-09-18,f,NaN,NaN,4,90
2,5456,2025-09-19,f,NaN,NaN,4,90
3,5456,2025-09-20,f,NaN,NaN,4,90
4,5456,2025-09-21,t,NaN,NaN,4,90


In [5]:
calendar.describe()

,listing_id,price,adjusted_price,minimum_nights,maximum_nights
count,3.844547e+06,0.0,0.0,3.844547e+06,3.844547e+06
mean,7.673740e+17,NaN,NaN,8.303395e+00,8.469798e+06
std,5.572091e+17,NaN,NaN,2.095058e+01,1.345945e+08
min,5.456000e+03,NaN,NaN,1.000000e+00,1.000000e+00
25%,5.060361e+07,NaN,NaN,2.000000e+00,3.650000e+02
50%,8.952778e+17,NaN,NaN,2.000000e+00,3.650000e+02
75%,1.260918e+18,NaN,NaN,3.000000e+00,1.125000e+03
max,1.510543e+18,NaN,NaN,9.150000e+02,2.147484e+09


In [6]:
calendar.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3844547 entries, 0 to 3844546
Data columns (total 7 columns):
 #   Column          Dtype  
---  ------          -----  
 0   listing_id      int64  
 1   date            object 
 2   available       object 
 3   price           float64
 4   adjusted_price  float64
 5   minimum_nights  int64  
 6   maximum_nights  int64  
dtypes: float64(2), int64(3), object(2)
memory usage: 205.3+ MB


In [7]:
calendar.isna().sum()

listing_id              0
date                    0
available               0
price             3844547
adjusted_price    3844547
minimum_nights          0
maximum_nights          0
dtype: int64

In [8]:
calendar['available'] = (
    calendar['available']
    .map({'t': 1, 'f': 0})
    .astype('Int64')
)


In [9]:
non_null_counts = calendar.notna().sum()
non_null_counts

listing_id        3844547
date              3844547
available         3844547
price                   0
adjusted_price          0
minimum_nights    3844547
maximum_nights    3844547
dtype: int64

In [10]:
# 1. Ensure correct data types
calendar['date'] = pd.to_datetime(calendar['date'], errors='coerce')

calendar['available'] = calendar['available'].astype(int)

calendar['minimum_nights'] = calendar['minimum_nights'].astype(int)
calendar['maximum_nights'] = calendar['maximum_nights'].astype(int)

In [11]:
#Visualize Outliers
calendar[['minimum_nights', 'maximum_nights']].quantile([0.95, 0.99])


,minimum_nights,maximum_nights
0.95,30.0,1125.0
0.99,90.0,1125.0


In [12]:
#Define Business Thresholds
MIN_NIGHTS_CAP = 90
MAX_NIGHTS_CAP = 1125


In [13]:
#Clean minimum_nights
calendar['minimum_nights_clean'] = calendar['minimum_nights'].clip(
    lower=1,
    upper=MIN_NIGHTS_CAP
)


In [14]:
#Clean maximum_nights
calendar['maximum_nights_clean'] = calendar['maximum_nights'].clip(
    lower=calendar['minimum_nights_clean'],
    upper=MAX_NIGHTS_CAP
)

In [15]:
#count= calendar[calendar[ 'maximum_nights_clean']>1125]
#count

In [16]:
calendar[['minimum_nights', 'maximum_nights','minimum_nights_clean', 'maximum_nights_clean']].describe()

,minimum_nights,maximum_nights,minimum_nights_clean,maximum_nights_clean
count,3.844547e+06,3.844547e+06,3.844547e+06,3.844547e+06
mean,8.303395e+00,8.469798e+06,7.608384e+00,6.259394e+02
std,2.095058e+01,1.345945e+08,1.386686e+01,4.530474e+02
min,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
25%,2.000000e+00,3.650000e+02,2.000000e+00,3.650000e+02
50%,2.000000e+00,3.650000e+02,2.000000e+00,3.650000e+02
75%,3.000000e+00,1.125000e+03,3.000000e+00,1.125000e+03
max,9.150000e+02,2.147484e+09,9.000000e+01,1.125000e+03


MIN_NIGHTS_CAP = 90

calendar = calendar[
    calendar['minimum_nights'] <= MIN_NIGHTS_CAP
]


In [17]:
calendar= calendar.drop(columns=['price', 'adjusted_price'])

In [18]:
calendar.notnull().sum()

listing_id              3844547
date                    3844547
available               3844547
minimum_nights          3844547
maximum_nights          3844547
minimum_nights_clean    3844547
maximum_nights_clean    3844547
dtype: int64

In [19]:
calendar.to_parquet('/kaggle/working/1cleaned_calendar.parquet')

In [20]:
# listings Table – Data Cleaning

In [21]:
listings.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5456,https://www.airbnb.com/rooms/5456,20250916040734,2025-09-17,city scrape,"Walk to 6th, Rainey St and Convention Ctr",Great central location for walking to Convent...,My neighborhood is ideally located if you want...,https://a0.muscache.com/pictures/14084884/b5a3...,8028,...,4.82,4.73,4.79,NaN,f,1,1,0,0,3.52
1,6448,https://www.airbnb.com/rooms/6448,20250916040734,2025-09-17,city scrape,"Secluded Studio @ Zilker - King Bed, Bright & ...","Clean, private space with everything you need ...",The neighborhood is fun and funky (but quiet)!...,https://a0.muscache.com/pictures/airflow/Hosti...,14156,...,4.98,4.97,4.88,NaN,t,1,1,0,0,1.98
2,8502,https://www.airbnb.com/rooms/8502,20250916040734,2025-09-17,city scrape,Woodland Studio Lodging,Studio rental on lower level of home located i...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,25298,...,4.88,4.69,4.63,NaN,f,1,1,0,0,0.28
3,13035,https://www.airbnb.com/rooms/13035,20250916040734,2025-09-17,city scrape,Historic house in highly walkable East Austin,Comfortable 2 bedroom/2 bathroom home very cen...,East Cesar Chavez is a gentrifying urban area ...,https://a0.muscache.com/pictures/miso/Hosting-...,50793,...,5.00,5.00,4.95,NaN,f,2,2,0,0,0.11
4,22828,https://www.airbnb.com/rooms/22828,20250916040734,2025-09-16,city scrape,Garage Apartment central SE Austin,"Fully furnished, centrally located, second sto...","wikipedia: East_Riverside-Oltorf,_Austin,_Texas",https://a0.muscache.com/pictures/miso/Hosting-...,56488,...,5.00,4.72,4.84,NaN,f,1,1,0,0,0.30


In [22]:
listings.describe()

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_cleansed,neighbourhood_group_cleansed,latitude,longitude,accommodates,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,1.053300e+04,1.053300e+04,1.053300e+04,10524.000000,10524.000000,10533.000000,0.0,10533.000000,10533.000000,10533.000000,...,8909.000000,8909.000000,8909.000000,8909.000000,0.0,10533.000000,10533.000000,10533.000000,10533.000000,8909.000000
mean,7.673744e+17,2.025092e+13,2.051309e+08,114.136830,165.268814,78724.782683,NaN,30.281656,-97.749225,5.334188,...,4.894308,4.902264,4.825561,4.778510,NaN,9.856736,8.208677,1.005507,0.520554,1.742838
std,5.572353e+17,1.828212e+00,2.125070e+08,563.994257,787.656139,20.868113,NaN,0.065631,0.064934,3.636149,...,0.277543,0.264418,0.284337,0.328332,NaN,17.808890,15.960760,4.483837,5.553189,1.809563
min,5.456000e+03,2.025092e+13,2.300000e+01,1.000000,1.000000,78701.000000,NaN,30.078440,-98.053350,1.000000,...,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,0.000000,0.000000,0.000000,0.010000
25%,5.060361e+07,2.025092e+13,2.192214e+07,1.000000,2.000000,78704.000000,NaN,30.242391,-97.767457,2.000000,...,4.900000,4.900000,4.780000,4.730000,NaN,1.000000,1.000000,0.000000,0.000000,0.430000
50%,8.952778e+17,2.025092e+13,1.091668e+08,3.000000,5.000000,78723.000000,NaN,30.269836,-97.739867,4.000000,...,4.970000,4.980000,4.900000,4.850000,NaN,2.000000,2.000000,0.000000,0.000000,1.230000
75%,1.260918e+18,2.025092e+13,3.972112e+08,18.000000,26.000000,78745.000000,NaN,30.313840,-97.715320,6.000000,...,5.000000,5.000000,5.000000,4.950000,NaN,9.000000,7.000000,0.000000,0.000000,2.510000
max,1.510543e+18,2.025092e+13,7.181423e+08,5030.000000,9684.000000,78759.000000,NaN,30.519400,-97.562440,16.000000,...,5.000000,5.000000,5.000000,5.000000,NaN,96.000000,96.000000,42.000000,60.000000,41.140000


In [23]:
listings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10533 entries, 0 to 10532
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            10533 non-null  int64  
 1   listing_url                                   10533 non-null  object 
 2   scrape_id                                     10533 non-null  int64  
 3   last_scraped                                  10533 non-null  object 
 4   source                                        10533 non-null  object 
 5   name                                          10533 non-null  object 
 6   description                                   10311 non-null  object 
 7   neighborhood_overview                         5360 non-null   object 
 8   picture_url                                   10533 non-null  object 
 9   host_id                                       10533 non-null 

In [24]:
listings

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,5456,https://www.airbnb.com/rooms/5456,20250916040734,2025-09-17,city scrape,"Walk to 6th, Rainey St and Convention Ctr",Great central location for walking to Convent...,My neighborhood is ideally located if you want...,https://a0.muscache.com/pictures/14084884/b5a3...,8028,...,4.82,4.73,4.79,NaN,f,1,1,0,0,3.52
1,6448,https://www.airbnb.com/rooms/6448,20250916040734,2025-09-17,city scrape,"Secluded Studio @ Zilker - King Bed, Bright & ...","Clean, private space with everything you need ...",The neighborhood is fun and funky (but quiet)!...,https://a0.muscache.com/pictures/airflow/Hosti...,14156,...,4.98,4.97,4.88,NaN,t,1,1,0,0,1.98
2,8502,https://www.airbnb.com/rooms/8502,20250916040734,2025-09-17,city scrape,Woodland Studio Lodging,Studio rental on lower level of home located i...,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,25298,...,4.88,4.69,4.63,NaN,f,1,1,0,0,0.28
3,13035,https://www.airbnb.com/rooms/13035,20250916040734,2025-09-17,city scrape,Historic house in highly walkable East Austin,Comfortable 2 bedroom/2 bathroom home very cen...,East Cesar Chavez is a gentrifying urban area ...,https://a0.muscache.com/pictures/miso/Hosting-...,50793,...,5.00,5.00,4.95,NaN,f,2,2,0,0,0.11
4,22828,https://www.airbnb.com/rooms/22828,20250916040734,2025-09-16,city scrape,Garage Apartment central SE Austin,"Fully furnished, centrally located, second sto...","wikipedia: East_Riverside-Oltorf,_Austin,_Texas",https://a0.muscache.com/pictures/miso/Hosting-...,56488,...,5.00,4.72,4.84,NaN,f,1,1,0,0,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10528,1509111840521525342,https://www.airbnb.com/rooms/1509111840521525342,20250916040734,2025-09-16,city scrape,East Austin Escape | Spacious Stylish Home w/G...,Enjoy your stay in this modern-style East Aust...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,478009344,...,NaN,NaN,NaN,NaN,f,19,19,0,0,NaN
10529,1509788889374663874,https://www.airbnb.com/rooms/1509788889374663874,20250916040734,2025-09-16,city scrape,King Bedroom | Shared Bath & Amenities,Spacious king bedroom with Smart TV and dedica...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,115683639,...,NaN,NaN,NaN,NaN,t,7,5,2,0,NaN
10530,1510032268538948494,https://www.airbnb.com/rooms/1510032268538948494,20250916040734,2025-09-17,city scrape,Tranquilo y comodo,Enjoy the simplicity of this quiet and central...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,301397512,...,NaN,NaN,NaN,NaN,f,2,0,2,0,NaN
10531,1510421017787311377,https://www.airbnb.com/rooms/1510421017787311377,20250916040734,2025-09-16,city scrape,Quaint Clarksville Condo,Charming studio apartment in historic Clarksvi...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,36290514,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN


In [25]:
drop_cols = [
    'listing_url','scrape_id','source','picture_url',
    'host_url','host_thumbnail_url','host_picture_url',
    'neighbourhood','neighbourhood_group_cleansed',
    'bathrooms_text','calendar_updated','has_availability',
    'license'
]

listings.drop(columns=drop_cols, inplace=True)


In [26]:
date_cols = [
    'last_scraped','host_since','calendar_last_scraped',
    'first_review','last_review'
]

for col in date_cols:
    listings[col] = pd.to_datetime(listings[col], errors='coerce')


In [27]:
bool_cols = [
    'host_is_superhost','host_has_profile_pic',
    'host_identity_verified','instant_bookable'
]

for col in bool_cols:
    listings[col] = listings[col].map({'t':1, 'f':0})


In [28]:
pct_cols = ['host_response_rate','host_acceptance_rate']

for col in pct_cols:
    listings[col] = (
        listings[col]
        .str.replace('%','', regex=False)
        .astype(float)
    )


In [29]:
listings['price'] = (
    listings['price']
    .str.replace('[$,]','', regex=True)
    .astype(float)
)


In [30]:
num_fill = ['bathrooms', 'bedrooms', 'beds']

for col in num_fill:
    listings[col] = listings[col].fillna(listings[col].median())


In [31]:
MIN_NIGHTS_CAP = 365
MAX_NIGHTS_CAP = 1825


In [32]:
listings['minimum_nights'] = listings['minimum_nights'].clip(1, MIN_NIGHTS_CAP)
listings['maximum_nights'] = listings['maximum_nights'].clip(1, MAX_NIGHTS_CAP)


In [33]:
night_cols = [
    'minimum_maximum_nights','maximum_maximum_nights',
    'maximum_nights_avg_ntm'
]

for col in night_cols:
    listings.loc[listings[col] > 1_000_000, col] = np.nan


In [34]:
listings = listings[listings['amenities'] != '[]']


In [35]:
listings.info()
listings.describe(include='all')


<class 'pandas.core.frame.DataFrame'>
Index: 10531 entries, 0 to 10532
Data columns (total 66 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   id                                            10531 non-null  int64         
 1   last_scraped                                  10531 non-null  datetime64[ns]
 2   name                                          10531 non-null  object        
 3   description                                   10309 non-null  object        
 4   neighborhood_overview                         5360 non-null   object        
 5   host_id                                       10531 non-null  int64         
 6   host_name                                     10522 non-null  object        
 7   host_since                                    10522 non-null  datetime64[ns]
 8   host_location                                 8656 non-null   object   

,id,last_scraped,name,description,neighborhood_overview,host_id,host_name,host_since,host_location,host_about,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,1.053100e+04,10531,10531,10309,5360,1.053100e+04,10522,10522,8656,6958,...,8909.000000,8909.000000,8909.000000,8909.000000,10531.000000,10531.000000,10531.000000,10531.000000,10531.000000,8909.000000
unique,NaN,NaN,10296,9140,4152,NaN,2588,NaN,356,3137,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
top,NaN,NaN,Wyndham Austin Resort|1BR/1BA King Bed w/ Gym&...,Please click on -show more- and read the descr...,"• CW Austin Resort is in Austin, Texas.",NaN,Vacasa Texas,NaN,"Austin, TX",Vacasa\nVacation Home Management\n\nVacasa unl...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
freq,NaN,NaN,40,42,112,NaN,131,NaN,6778,123,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
mean,7.674156e+17,2025-09-16 11:01:24.436425984,NaN,NaN,NaN,2.051586e+08,NaN,2017-10-24 00:31:36.825698304,NaN,NaN,...,4.894308,4.902264,4.825561,4.778510,0.411072,9.858418,8.210047,1.005697,0.520653,1.742838
min,5.456000e+03,2025-09-16 00:00:00,NaN,NaN,NaN,2.300000e+01,NaN,2008-03-03 00:00:00,NaN,NaN,...,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.010000
25%,5.060465e+07,2025-09-16 00:00:00,NaN,NaN,NaN,2.190631e+07,NaN,2014-10-01 00:00:00,NaN,NaN,...,4.900000,4.900000,4.780000,4.730000,0.000000,1.000000,1.000000,0.000000,0.000000,0.430000
50%,8.952778e+17,2025-09-16 00:00:00,NaN,NaN,NaN,1.094553e+08,NaN,2017-01-02 00:00:00,NaN,NaN,...,4.970000,4.980000,4.900000,4.850000,0.000000,2.000000,2.000000,0.000000,0.000000,1.230000
75%,1.260939e+18,2025-09-17 00:00:00,NaN,NaN,NaN,3.973055e+08,NaN,2021-04-20 18:00:00,NaN,NaN,...,5.000000,5.000000,5.000000,4.950000,1.000000,9.000000,7.000000,0.000000,0.000000,2.510000
max,1.510543e+18,2025-09-17 00:00:00,NaN,NaN,NaN,7.181423e+08,NaN,2025-09-09 00:00:00,NaN,NaN,...,5.000000,5.000000,5.000000,5.000000,1.000000,96.000000,96.000000,42.000000,60.000000,41.140000


In [36]:
#Fix encoding artifacts
text_cols = ['name','description','neighborhood_overview','host_about']

for col in text_cols:
    listings[col] = (
        listings[col]
        .str.encode('latin1', errors='ignore')
        .str.decode('utf-8', errors='ignore')
    )


/tmp/ipykernel_55/2378765132.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings[col] = (


In [46]:
#Your latitude/longitude should be Austin-bounded.
listings = listings[
    listings['latitude'].between(30.0, 30.6) &
    listings['longitude'].between(-98.0, -97.5)
]


In [47]:
#Normalize Amenities
listings['amenities_count'] = listings['amenities'].apply(
    lambda x: len(eval(x)) if isinstance(x, str) else 0
)


In [48]:
listings.columns

Index(['id', 'last_scraped', 'name', 'description', 'neighborhood_overview',
       'host_id', 'host_name', 'host_since', 'host_location', 'host_about',
       'host_response_time', 'host_response_rate', 'host_acceptance_rate',
       'host_is_superhost', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified',
       'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'amenities', 'price', 'minimum_nights', 'maximum_nights',
       'minimum_minimum_nights', 'maximum_minimum_nights',
       'minimum_maximum_nights', 'maximum_maximum_nights',
       'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm', 'availability_30',
       'availability_60', 'availability_90', 'availability_365',
       'calendar_last_scraped', 'number_of_reviews', 'number_of_reviews_ltm',
       'number_of_reviews_l30d', 'av

In [49]:
#Host Portfolio Consistency Check
listings = listings[
    listings['calculated_host_listings_count'] ==
    listings[['calculated_host_listings_count_entire_homes',
              'calculated_host_listings_count_private_rooms',
              'calculated_host_listings_count_shared_rooms']].sum(axis=1)
]
listings

,id,last_scraped,name,description,neighborhood_overview,host_id,host_name,host_since,host_location,host_about,...,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month,amenities_count
0,5456,2025-09-17,"Walk to 6th, Rainey St and Convention Ctr",Great central location for walking to Convent...,My neighborhood is ideally located if you want...,8028,Sylvia,2009-02-16,"Austin, TX",I am a licensed Real Estate Broker and owner o...,...,4.82,4.73,4.79,0,1,1,0,0,3.52,26
1,6448,2025-09-17,"Secluded Studio @ Zilker - King Bed, Bright & ...","Clean, private space with everything you need ...",The neighborhood is fun and funky (but quiet)!...,14156,Amy,2009-04-20,"Austin, TX","We are a family of four (with teenagers, all o...",...,4.98,4.97,4.88,1,1,1,0,0,1.98,61
2,8502,2025-09-17,Woodland Studio Lodging,Studio rental on lower level of home located i...,NaN,25298,Karen,2009-07-11,"Austin, TX",I handle the reservations at the studio on the...,...,4.88,4.69,4.63,0,1,1,0,0,0.28,12
3,13035,2025-09-17,Historic house in highly walkable East Austin,Comfortable 2 bedroom/2 bathroom home very cen...,East Cesar Chavez is a gentrifying urban area ...,50793,Margaret Ann,2009-11-02,"Austin, TX","We're a responsible, easygoing couple who enjo...",...,5.00,5.00,4.95,0,2,2,0,0,0.11,54
4,22828,2025-09-16,Garage Apartment central SE Austin,"Fully furnished, centrally located, second sto...","wikipedia: East_Riverside-Oltorf,_Austin,_Texas",56488,David,2009-11-22,"Austin, TX",Wyoming native living in Austin since 1996. I ...,...,5.00,4.72,4.84,0,1,1,0,0,0.30,43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10528,1509111840521525342,2025-09-16,East Austin Escape | Spacious Stylish Home w/G...,Enjoy your stay in this modern-style East Aust...,NaN,478009344,Paul,2022-09-03,NaN,NaN,...,NaN,NaN,NaN,0,19,19,0,0,NaN,37
10529,1509788889374663874,2025-09-16,King Bedroom | Shared Bath & Amenities,Spacious king bedroom with Smart TV and dedica...,NaN,115683639,Dayna,2017-02-09,"Pflugerville, TX",NaN,...,NaN,NaN,NaN,1,7,5,2,0,NaN,13
10530,1510032268538948494,2025-09-17,Tranquilo y comodo,Enjoy the simplicity of this quiet and central...,NaN,301397512,Luis Renan,2019-10-10,"Austin, TX",NaN,...,NaN,NaN,NaN,0,2,0,2,0,NaN,13
10531,1510421017787311377,2025-09-16,Quaint Clarksville Condo,Charming studio apartment in historic Clarksvi...,NaN,36290514,John,2015-06-20,"Austin, TX",NaN,...,NaN,NaN,NaN,0,1,1,0,0,NaN,33


In [50]:
lower = listings['price'].quantile(0.01)
upper = listings['price'].quantile(0.99)

listings = listings[listings['price'].between(lower, upper)]


In [51]:
listings['price'].describe()

count    9946.000000
mean      188.210235
std       169.282207
min        33.000000
25%        87.000000
50%       134.000000
75%       216.000000
max      1149.000000
Name: price, dtype: float64

In [52]:
listings.to_parquet("/kaggle/working/cleaned_listings1.parquet")


In [ ]:
# 3) cleaning listings_extra table

In [50]:
listings_extra.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
0,5456,"Walk to 6th, Rainey St and Convention Ctr",8028,Sylvia,NaN,78702,30.26057,-97.73441,Entire home/apt,97.0,2,708,2025-09-02,3.52,1,328,25,NaN
1,6448,"Secluded Studio @ Zilker - King Bed, Bright & ...",14156,Amy,NaN,78704,30.26034,-97.76487,Entire home/apt,160.0,3,339,2025-08-20,1.98,1,316,14,NaN
2,8502,Woodland Studio Lodging,25298,Karen,NaN,78741,30.23466,-97.73682,Entire home/apt,38.0,4,54,2025-05-05,0.28,1,88,1,NaN
3,13035,Historic house in highly walkable East Austin,50793,Margaret Ann,NaN,78702,30.26098,-97.73072,Entire home/apt,145.0,15,19,2025-07-31,0.11,2,321,1,NaN
4,22828,Garage Apartment central SE Austin,56488,David,NaN,78741,30.23614,-97.73225,Entire home/apt,58.0,30,56,2025-08-16,0.30,1,211,3,NaN


In [51]:
#initial inspection
listings_extra.info()
listings_extra.describe(include='all')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10533 entries, 0 to 10532
Data columns (total 18 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              10533 non-null  int64  
 1   name                            10533 non-null  object 
 2   host_id                         10533 non-null  int64  
 3   host_name                       10524 non-null  object 
 4   neighbourhood_group             0 non-null      float64
 5   neighbourhood                   10533 non-null  int64  
 6   latitude                        10533 non-null  float64
 7   longitude                       10533 non-null  float64
 8   room_type                       10533 non-null  object 
 9   price                           10517 non-null  float64
 10  minimum_nights                  10533 non-null  int64  
 11  number_of_reviews               10533 non-null  int64  
 12  last_review                     

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,number_of_reviews_ltm,license
count,1.053300e+04,10533,1.053300e+04,10524,0.0,10533.000000,10533.000000,10533.000000,10533,10517.000000,10533.000000,10533.000000,8909,8909.000000,10533.000000,10533.000000,10533.000000,0.0
unique,NaN,10298,NaN,2589,NaN,NaN,NaN,NaN,4,NaN,NaN,NaN,865,NaN,NaN,NaN,NaN,NaN
top,NaN,Wyndham Austin Resort|1BR/1BA King Bed w/ Gym&...,NaN,Vacasa Texas,NaN,NaN,NaN,NaN,Entire home/apt,NaN,NaN,NaN,2025-09-01,NaN,NaN,NaN,NaN,NaN
freq,NaN,40,NaN,131,NaN,NaN,NaN,NaN,9078,NaN,NaN,NaN,716,NaN,NaN,NaN,NaN,NaN
mean,7.673744e+17,NaN,2.051309e+08,NaN,NaN,78724.782683,30.281656,-97.749225,NaN,414.537701,7.768157,55.858920,NaN,1.742838,9.856736,234.909238,13.065698,NaN
std,5.572353e+17,NaN,2.125070e+08,NaN,NaN,20.868113,0.065631,0.064934,NaN,2896.807897,20.586238,104.276866,NaN,1.809563,17.808890,114.102467,18.844388,NaN
min,5.456000e+03,NaN,2.300000e+01,NaN,NaN,78701.000000,30.078440,-98.053350,NaN,8.000000,1.000000,0.000000,NaN,0.010000,1.000000,0.000000,0.000000,NaN
25%,5.060361e+07,NaN,2.192214e+07,NaN,NaN,78704.000000,30.242391,-97.767457,NaN,86.000000,1.000000,2.000000,NaN,0.430000,1.000000,143.000000,1.000000,NaN
50%,8.952778e+17,NaN,1.091668e+08,NaN,NaN,78723.000000,30.269836,-97.739867,NaN,135.000000,2.000000,17.000000,NaN,1.230000,2.000000,263.000000,5.000000,NaN
75%,1.260918e+18,NaN,3.972112e+08,NaN,NaN,78745.000000,30.313840,-97.715320,NaN,226.000000,3.000000,62.000000,NaN,2.510000,9.000000,339.000000,19.000000,NaN


In [52]:
# Drop Non-Informative Columns
listings_extra.drop(
    columns=['neighbourhood_group', 'license'],
    inplace=True
)


In [53]:
#Handle Missing Categorical Values
listings_extra['host_name'].fillna('Unknown', inplace=True)

/tmp/ipykernel_55/3065443695.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  listings_extra['host_name'].fillna('Unknown', inplace=True)


In [54]:
#Convert Data Types
listings_extra['last_review'] = pd.to_datetime(
    listings_extra['last_review'], errors='coerce'
)


In [55]:
listings_extra = listings_extra[listings_extra['price'] > 0]
lower_cap = listings_extra['price'].quantile(0.01)
upper_cap = listings_extra['price'].quantile(0.99)
listings_extra['price'] = listings_extra['price'].clip(lower=lower_cap, upper=upper_cap)

In [56]:
listings_extra.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10517 entries, 0 to 10532
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              10517 non-null  int64         
 1   name                            10517 non-null  object        
 2   host_id                         10517 non-null  int64         
 3   host_name                       10517 non-null  object        
 4   neighbourhood                   10517 non-null  int64         
 5   latitude                        10517 non-null  float64       
 6   longitude                       10517 non-null  float64       
 7   room_type                       10517 non-null  object        
 8   price                           10517 non-null  float64       
 9   minimum_nights                  10517 non-null  int64         
 10  number_of_reviews               10517 non-null  int64         
 11  last_re

In [65]:
#Handle Review-Related Missing Values
listings_extra['reviews_per_month'].fillna(0, inplace=True)
listings_extra['has_reviews'] = listings_extra['last_review'].notna().astype(int)

/tmp/ipykernel_55/3594629934.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  listings_extra['reviews_per_month'].fillna(0, inplace=True)


In [66]:
listings_extra = listings_extra[
    (listings_extra['minimum_nights'] >= 1) &
    (listings_extra['minimum_nights'] <= 365)
]

listings_extra = listings_extra[
    listings_extra['availability_365'].between(0, 365)
]


In [67]:
listings_extra = listings_extra[
    listings_extra['latitude'].between(29, 31) &
    listings_extra['longitude'].between(-99, -96)
]

In [68]:
listings_extra.drop_duplicates(subset='id', inplace=True)


In [69]:
listings_extra.info()
listings_extra.isna().sum()

<class 'pandas.core.frame.DataFrame'>
Index: 10517 entries, 0 to 10532
Data columns (total 17 columns):
 #   Column                          Non-Null Count  Dtype         
---  ------                          --------------  -----         
 0   id                              10517 non-null  int64         
 1   name                            10517 non-null  object        
 2   host_id                         10517 non-null  int64         
 3   host_name                       10517 non-null  object        
 4   neighbourhood                   10517 non-null  int64         
 5   latitude                        10517 non-null  float64       
 6   longitude                       10517 non-null  float64       
 7   room_type                       10517 non-null  object        
 8   price                           10517 non-null  float64       
 9   minimum_nights                  10517 non-null  int64         
 10  number_of_reviews               10517 non-null  int64         
 11  last_re

id                                   0
name                                 0
host_id                              0
host_name                            0
neighbourhood                        0
latitude                             0
longitude                            0
room_type                            0
price                                0
minimum_nights                       0
number_of_reviews                    0
last_review                       1616
reviews_per_month                    0
calculated_host_listings_count       0
availability_365                     0
number_of_reviews_ltm                0
has_reviews                          0
dtype: int64

In [70]:
listings_extra.to_parquet("cleaned_listings2.parquet")

In [ ]:
# 4)cleaning reviews table

In [4]:
reviews.head()

,listing_id,id,date,reviewer_id,reviewer_name,comments
0,5456,977,2009-03-19,8102,Phil,Highly recommended. Sylvia was extremely helpf...
1,5456,1039,2009-03-22,8241,Galen,A great place to stay in a great city. Sylvia ...
2,5456,1347,2009-04-08,11152,April,Highly recommended! \r<br/>Cute and cozy guest...
3,5456,1491,2009-04-13,12400,Ivonne,"What a great little apartment! It was clean, ..."
4,5456,1535,2009-04-16,11071,Egan.Sturges.Regan,"Sylvia was great; ""ditto"" to all the previous ..."


In [5]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 588362 entries, 0 to 588361
Data columns (total 6 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   listing_id     588362 non-null  int64 
 1   id             588362 non-null  int64 
 2   date           588362 non-null  object
 3   reviewer_id    588362 non-null  int64 
 4   reviewer_name  588361 non-null  object
 5   comments       588182 non-null  object
dtypes: int64(3), object(3)
memory usage: 26.9+ MB


In [30]:
reviews.describe(include='all')

,listing_id,id,date,reviewer_id,reviewer_name,comments
count,5.883620e+05,5.883620e+05,588362,5.883620e+05,588362.0,588182
unique,NaN,NaN,NaN,NaN,NaN,562806
top,NaN,NaN,NaN,NaN,NaN,Great stay!
freq,NaN,NaN,NaN,NaN,NaN,1120
mean,3.037074e+17,7.544972e+17,2022-07-16 02:47:19.141889280,1.902465e+08,1.0,NaN
min,5.456000e+03,9.770000e+02,2009-03-19 00:00:00,1.400000e+01,1.0,NaN
25%,1.710393e+07,7.379531e+08,2021-03-10 00:00:00,4.362381e+07,1.0,NaN
50%,4.193594e+07,8.710032e+17,2023-04-16 00:00:00,1.318087e+08,1.0,NaN
75%,6.984276e+17,1.225503e+18,2024-08-17 00:00:00,3.134046e+08,1.0,NaN
max,1.507609e+18,1.511781e+18,2025-09-16 00:00:00,7.188310e+08,1.0,NaN


In [31]:
reviews['date'] = pd.to_datetime(reviews['date'], errors='coerce')

In [32]:
#Clean Review Text (comments)
reviews['comments'] = (
    reviews['comments']
    .astype(str)
    .str.replace(r'<.*?>', '', regex=True)
    .str.strip()
)


In [34]:
# downloading the clean table
reviews.to_parquet("cleaned_reviews1.parquet")


In [39]:
# 5)cleaning reviews_extra table

In [40]:
reviews_extra.head()

,listing_id,date
0,5456,2009-03-19
1,5456,2009-03-22
2,5456,2009-04-08
3,5456,2009-04-13
4,5456,2009-04-16


In [53]:
reviews_extra.info()

<class 'pandas.core.frame.DataFrame'>
Index: 584430 entries, 0 to 588361
Data columns (total 2 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   listing_id  584430 non-null  int64 
 1   date        584430 non-null  object
dtypes: int64(1), object(1)
memory usage: 13.4+ MB


In [41]:
reviews_extra.describe(include='all')

,listing_id,date
count,5.883620e+05,588362
unique,NaN,5138
top,NaN,2025-04-06
freq,NaN,1338
mean,3.037074e+17,NaN
std,4.461331e+17,NaN
min,5.456000e+03,NaN
25%,1.710393e+07,NaN
50%,4.193594e+07,NaN
75%,6.984276e+17,NaN


In [50]:
# Inspect duplicates
reviews_extra.duplicated().sum()

np.int64(0)

In [52]:
#Drop full duplicates
reviews_extra = reviews_extra.drop_duplicates()

In [55]:
reviews_extra = reviews_extra[
    reviews_extra['listing_id'].notna() &
    (reviews_extra['listing_id'] > 0)
]


In [58]:
reviews_extra.to_parquet(
    "Cleaned_Reviews_Extra.parquet",
    index=False
)

In [9]:
# 6)cleaning neighbourhoods table


In [5]:
neighbourhoods.head()

,neighbourhood_group,neighbourhood
0,NaN,78701
1,NaN,78702
2,NaN,78703
3,NaN,78704
4,NaN,78705


In [6]:
neighbourhoods.describe()

,neighbourhood_group,neighbourhood
count,0.0,44.000000
mean,NaN,78733.750000
std,NaN,16.421944
min,NaN,78701.000000
25%,NaN,78723.750000
50%,NaN,78734.500000
75%,NaN,78747.250000
max,NaN,78759.000000


In [7]:
neighbourhoods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 2 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   neighbourhood_group  0 non-null      float64
 1   neighbourhood        44 non-null     int64  
dtypes: float64(1), int64(1)
memory usage: 836.0 bytes


In [8]:
neighbourhoods=neighbourhoods.drop(columns='neighbourhood_group')

In [9]:
neighbourhoods.to_parquet("cleaned_neighbourhoods.parquet", index=False)

In [10]:
# 7) Cleaning neighbourhoods_geo

In [11]:
neighbourhoods_geo.head()

,neighbourhood,neighbourhood_group,geometry
0,78739,None,"MULTIPOLYGON (((-97.89002 30.20941, -97.88929 ..."
1,78754,None,"MULTIPOLYGON (((-97.63658 30.40265, -97.63754 ..."
2,78732,None,"MULTIPOLYGON (((-97.87303 30.43806, -97.87335 ..."
3,78737,None,"MULTIPOLYGON (((-98.01876 30.24166, -98.01617 ..."
4,78756,None,"MULTIPOLYGON (((-97.72793 30.32905, -97.72761 ..."


In [12]:
neighbourhoods_geo.describe(include='all')

,neighbourhood,neighbourhood_group,geometry
count,44,0,44
unique,44,0,44
top,78739,NaN,"MULTIPOLYGON (((-97.89002 30.209408, -97.88929..."
freq,1,NaN,1


In [13]:
neighbourhoods_geo.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype   
---  ------               --------------  -----   
 0   neighbourhood        44 non-null     object  
 1   neighbourhood_group  0 non-null      object  
 2   geometry             44 non-null     geometry
dtypes: geometry(1), object(2)
memory usage: 1.2+ KB


In [14]:
#Drop non-informative column
neighbourhoods_geo = neighbourhoods_geo.drop(columns=['neighbourhood_group'])

In [15]:
# Enforce correct data types
neighbourhoods_geo['neighbourhood'] = neighbourhoods_geo['neighbourhood'].astype(int)

In [16]:
#Validate geometry integrity
neighbourhoods_geo = neighbourhoods_geo[neighbourhoods_geo.is_valid]

In [17]:
#Set CRS (Coordinate Reference System
neighbourhoods_geo = neighbourhoods_geo.set_crs(epsg=4326, inplace=False)

In [18]:
neighbourhoods_geo.to_parquet('cleaned_neighbourhoods_geo.parquet', index=False)